## Writing Reviews to Postgres from CSV

In [5]:

import csv
from time import time
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, JSON, String, Text, text, Date
# from models import Review
from credentials import POSTGRESQL_USER, POSTGRESQL_PASSWORD, POSTGRESQL_DB, POSTGRESQL_HOST

from datetime import datetime, timedelta

# import io
import json
import os
import glob
# import base64

In [6]:
def get_paths(pathname):
    return sorted([os.path.join(pathname, f) for f in os.listdir(pathname) if f.endswith(".csv")])

In [55]:
DATA_DIR = './data/vacuum_reviews/Stick Vacuums & Electric Brooms'
# DATA_DIR = './data/bed_pillow_reviews/1-Beckham/'
# DATA_DIR = './data/bed_pillow_reviews/2-down alt/'
all_files = get_paths(DATA_DIR)

In [56]:
all_files

['./data/vacuum_reviews/Stick Vacuums & Electric Brooms/01 BISSELL Featherweight Stick Lightweight Bagless Vacuum with Crevice Tool, 2033M, Black 2021-07-30.csv',
 './data/vacuum_reviews/Stick Vacuums & Electric Brooms/02 Eureka Flash Lightweight Stick Vacuum Cleaner, 15KPa Powerful Suction, 2 in 1 Corded Handheld Vac for Hard Floor and Carpet, Black 2021-07-30.csv',
 './data/vacuum_reviews/Stick Vacuums & Electric Brooms/03 EUREKA NEC222 HyperClean Cordless Vacuum Cleaner, Deep Ocean 2021-07-30.csv',
 './data/vacuum_reviews/Stick Vacuums & Electric Brooms/04 Dirt Devil Vibe 3-in-1 Vacuum Cleaner, Lightweight Corded Bagless Stick Vac with Handheld, SD20020, Red 2021-07-30.csv',
 './data/vacuum_reviews/Stick Vacuums & Electric Brooms/05 TOPPIN Stick Vacuum Cleaner Cordless - Tangle-Free 6 in 1 Powerful 12Kpa Suction Stick Vacuum, Lightweight and Large Capacity, Up to 28min Runtime, Ideal for Home Hard Floor Carpet Car Pet 2021-07-30.csv',
 './data/vacuum_reviews/Stick Vacuums & Electric

In [57]:
def extract_product_category(file_path):
    return os.path.normpath(file_path).split(os.sep)[-2]
extract_product_category(all_files[0])

'Stick Vacuums & Electric Brooms'

In [58]:
def extract_product_name(file_path):
    return ('_').join(os.path.splitext(os.path.split(file_path)[-1])[0].replace(',', '').split(' ')[:-1])

In [59]:
extract_product_name(all_files[1])

'02_Eureka_Flash_Lightweight_Stick_Vacuum_Cleaner_15KPa_Powerful_Suction_2_in_1_Corded_Handheld_Vac_for_Hard_Floor_and_Carpet_Black'

In [60]:
Base = declarative_base()

class Review(Base):
    __tablename__ = 'reviews'

    review_id = Column(Integer, primary_key=True, server_default=text("nextval('reviews_review_id_seq'::regclass)"))
    content = Column(Text, nullable=False)
    meta_data = Column(JSON)
    product_id = Column(String(255), nullable=False)
    variation = Column(String(80))
    review_date = Column(Date)
    rater_id = Column(String(80))
    product_category = Column(String(80))
    product_rating = Column(Integer)

In [61]:
engine = create_engine(f"postgresql+psycopg2://{POSTGRESQL_USER}:{POSTGRESQL_PASSWORD}@{POSTGRESQL_HOST}:5432/{POSTGRESQL_DB}")
Base.metadata.create_all(engine)

session = sessionmaker(bind=engine)
s = session()

In [62]:
# PRODUCT_ID = 'BISSELL Crosswave' # Rename according to product ID

try:
    for file in all_files:
        print(f'processing {file}')
#         with open(file) as fh:
        product_category = extract_product_category(file)
        product_id = extract_product_name(file)
        with open(file, encoding='utf-8-sig') as fh:
            csvreader = csv.reader(fh, delimiter=',')
            headers = []
            for idx, row in enumerate(csvreader):
                if idx == 0:
                    headers = row
                else:
                    item = dict(zip(headers, row))
#                     print(item)
                    entry_item = {
                        'product_id': product_id,
                        'product_category': product_category,
                        'variation': item['Variation'],
                        'content': item['Title'] + ' - ' + item['Body'],
                        'product_rating': item['Rating'],
                        'rater_id': item['Author'],
                        'review_date': item['Date'].split(' on ')[1],
                        'meta_data': {k:d for (k, d) in item.items() if k not in ['Variation', 'Body', 'Rating', 'Author', 'Date']}
                        }
                    new_review = Review(**entry_item)
                    try:
                        s.add(new_review)
                    except:
                        print(entry_item)
    s.commit()
except():
    s.rollback()
    s.close()
            

processing ./data/vacuum_reviews/Stick Vacuums & Electric Brooms/01 BISSELL Featherweight Stick Lightweight Bagless Vacuum with Crevice Tool, 2033M, Black 2021-07-30.csv
processing ./data/vacuum_reviews/Stick Vacuums & Electric Brooms/02 Eureka Flash Lightweight Stick Vacuum Cleaner, 15KPa Powerful Suction, 2 in 1 Corded Handheld Vac for Hard Floor and Carpet, Black 2021-07-30.csv
processing ./data/vacuum_reviews/Stick Vacuums & Electric Brooms/03 EUREKA NEC222 HyperClean Cordless Vacuum Cleaner, Deep Ocean 2021-07-30.csv
processing ./data/vacuum_reviews/Stick Vacuums & Electric Brooms/04 Dirt Devil Vibe 3-in-1 Vacuum Cleaner, Lightweight Corded Bagless Stick Vac with Handheld, SD20020, Red 2021-07-30.csv
processing ./data/vacuum_reviews/Stick Vacuums & Electric Brooms/05 TOPPIN Stick Vacuum Cleaner Cordless - Tangle-Free 6 in 1 Powerful 12Kpa Suction Stick Vacuum, Lightweight and Large Capacity, Up to 28min Runtime, Ideal for Home Hard Floor Carpet Car Pet 2021-07-30.csv
processing ./

## Basic Text Analysis

In [81]:
import pandas as pd
import numpy as np

# Text preprocessing/analysis
import re
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [73]:
def read_files(files, separator=','):
    """
    Takes a list of pathnames and individually reads then concats them into a single DataFrame which is returned.
    Can handle Excel files, csv, or delimiter separated text.
    """
    processed_files = []
    for file in files:
        if file.lower().endswith('.xlsx') or file.lower().endswith('.xls'):
            processed_files.append(pd.read_excel(file, index_col=None, header=0))
        elif file.lower().endswith('.csv'):
            processed_files.append(pd.read_csv(file, index_col=None, header=0))
        else:
            processed_files.append(pd.read_csv(file, sep=separator, index_col=None, header=0))
    completed_df = pd.concat(processed_files, ignore_index=True)
    return completed_df

In [74]:
pillow_reviews = read_files(all_files)

In [88]:
pillow_reviews['ReviewCountry'], pillow_reviews['ReviewDate'] = pillow_reviews['Date'].str.split(' on ', 1).str

pillow_reviews['ReviewDate'] = pd.to_datetime(pillow_reviews['ReviewDate'])

/Users/michaeldu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [7]:
# Contains Image OR Video
print(f"Reviews with EITHER Image or Video: {len(pillow_reviews[(pillow_reviews.Images!='-') | (pillow_reviews.Videos!='-')])}")

# Contains No Image Nor Video
print(f"Reviews with NO Image or Video: {len(pillow_reviews[(pillow_reviews.Images=='-') & (pillow_reviews.Videos=='-')])}")

Reviews with EITHER Image or Video: 310
Reviews with NO Image or Video: 10686


In [8]:
# EXPORT FOR TABLEAU USE
# pillow_reviews.to_csv('dataset/pillow_reviews_{}.csv'.format(re.sub(r'(-|:| )', '', str(datetime.now())[:-7])), encoding='utf_8_sig')

In [10]:
prod_1 = pillow_reviews[pillow_reviews['Variation'] == 'B01LYNW421']

In [89]:
prod_1_good = prod_1[prod_1['Rating'] >= 4]
# prod_1_ok = prod_1[prod_1['Rating'] == 3]
prod_1_bad = prod_1[prod_1['Rating'] <= 3]

In [90]:
prod_1_good_reviews = prod_1_good.reset_index()['Body']
# prod_1_ok_reviews = prod_1_ok.reset_index()['Body']
prod_1_bad_reviews = prod_1_bad.reset_index()['Body']

In [13]:
def summarise(pattern, strings, freq):
    """Summarise strings matching a pattern."""
    # Find matches
    compiled_pattern = re.compile(pattern)
    matches = [s for s in strings if compiled_pattern.search(s)]
    
    # Print volume and proportion of matches
    print("{} strings, that is {:.2%} of total".format(len(matches), len(matches)/ len(strings)))
    
    # Create list of tuples containing matches and their frequency
    output = [(s, freq[s]) for s in set(matches)]
    output.sort(key=lambda x:x[1], reverse=True)
    
    return output

In [28]:
def find_outlaw(word):
    """Find words that contain a same character 3+ times in a row."""
    is_outlaw = False
    for i, letter in enumerate(word):
        if i > 1:
            if word[i] == word[i-1] == word[i-2] and word[i].isalpha():
                is_outlaw = True
                break
    return is_outlaw

## AWS SDK Testing

In [150]:
# from credentials import ACCESS_KEY, SECRET_KEY, 

# comprehend = boto3.client(
#     service_name='comprehend', 
#     region_name='us-west-2',
#     aws_access_key_id=ACCESS_KEY,
#     aws_secret_access_key=SECRET_KEY)

In [1]:
# text = single_review

# print('Calling DetectKeyPhrases')
# print(json.dumps(comprehend.detect_key_phrases(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
# print('End of DetectKeyPhrases\n')